In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/briankalinowski/Desktop/Data/news_content_final.csv')
print(df.shape)
df.head()

(26227, 6)


,title,text,type,tokenized_headline,tokenized_content,valid_score
0,Muslims BUSTED They Stole Millions In Govt Ben...,Print They should pay all the back all the mon...,bias,muslims,somalis,0
1,Re Why Did Attorney General Loretta Lynch Plea...,Why Did Attorney General Loretta Lynch Plead T...,bias,loretta lynch,loretta lynch barracuda brigade iran loretta l...,0
2,BREAKING Weiner Cooperating With FBI On Hillar...,Red State Fox News Sunday reported this mornin...,bias,weiner fbi hillary email investigation,red state fox news anthony weiner fbi hillary ...,0
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,bias,donald j. trump,kayla mueller isis carl mueller donald trump,0
4,FANTASTIC! TRUMPS 7 POINT PLAN To Reform Healt...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,bias,notokes,house senate obamacare obamacare democrats oba...,0


In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from IPython.display import display
import base64
import string
import re
from collections import Counter
from time import time
# from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from nltk.corpus import stopwords
from sklearn.metrics import log_loss

In [7]:
import nltk
nltk.download('stopwords')
stopwords = stopwords.words('english')
sns.set_context('notebook')
print('Numpy version:', np.__version__)
print('Pandas version:', pd.__version__)
print('Seaborn version:', sns.__version__)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/briankalinowski/nltk_data...


Numpy version: 1.17.2
Pandas version: 0.25.2
Seaborn version: 0.9.0


[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26227 entries, 0 to 26226
Data columns (total 6 columns):
title                 26226 non-null object
text                  26227 non-null object
type                  26227 non-null object
tokenized_headline    26227 non-null object
tokenized_content     26227 non-null object
valid_score           26227 non-null int64
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


In [10]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [12]:
punctuations = string.punctuation

# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [13]:
# Cleanup text and make sure it retains original shape
print('Original training data shape: ', df['text'].shape)
train_cleaned = cleanup_text(df['text'], logging=True)
print('Cleaned up training data shape: ', train_cleaned.shape)

Original training data shape:  (26227,)
Processed 1000 out of 26227 documents.


KeyboardInterrupt: 

In [ ]:
# Parse documents and print some info
print('Parsing documents...')

start = time()

train_vec = []
for doc in nlp.pipe(train_cleaned, batch_size=500):
    if doc.has_vector:
        train_vec.append(doc.vector)
    # If doc doesn't have a vector, then fill it with zeros.
    else:
        train_vec.append(np.zeros((128,), dtype="float32"))
        
# train_vec = [doc.vector for doc in nlp.pipe(train_cleaned, batch_size=500)]
train_vec = np.array(train_vec)

end = time()

In [17]:
print('Total time passed parsing documents: {} seconds'.format(end - start))
print('Total number of documents parsed: {}'.format(len(train_vec)))
print('Number of words in first document: ', len(df['text'][0]))
print('Number of words in second document: ', len(df['text'][1]))
print('Size of vector embeddings: ', train_vec.shape[1])
print('Shape of vectors embeddings matrix: ', train_vec.shape)

Total time passed parsing documents: 1099.8297290802002 seconds
Total number of documents parsed: 26227
Number of words in first document:  498
Number of words in second document:  1761
Size of vector embeddings:  96
Shape of vectors embeddings matrix:  (26227, 96)


In [18]:
# Define function to preprocess text for a word2vec model
def cleanup_text_word2vec(docs, logging=False):
    sentences = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents" % (counter, len(docs)))
        # Disable tagger so that lemma_ of personal pronouns (I, me, etc) don't getted marked as "-PRON-"
        doc = nlp(doc, disable=['tagger'])
        # Grab lemmatized form of words and make lowercase
        doc = " ".join([tok.lemma_.lower() for tok in doc])
        # Split into sentences based on punctuation
        doc = re.split("[\.?!;] ", doc)
        # Remove commas, periods, and other punctuation (mostly commas)
        doc = [re.sub("[\.,;:!?]", "", sent) for sent in doc]
        # Split into words
        doc = [sent.split() for sent in doc]
        sentences += doc
        counter += 1
    return sentences

In [21]:
train_cleaned_word2vec = cleanup_text_word2vec(df['text'], logging=True)
print('Cleaned up training data size (i.e. number of sentences): ', len(train_cleaned_word2vec))

Processed 1000 out of 26227 documents
Processed 2000 out of 26227 documents
Processed 3000 out of 26227 documents
Processed 4000 out of 26227 documents
Processed 5000 out of 26227 documents
Processed 6000 out of 26227 documents
Processed 7000 out of 26227 documents
Processed 8000 out of 26227 documents
Processed 9000 out of 26227 documents
Processed 10000 out of 26227 documents
Processed 11000 out of 26227 documents
Processed 12000 out of 26227 documents
Processed 13000 out of 26227 documents
Processed 14000 out of 26227 documents
Processed 15000 out of 26227 documents
Processed 16000 out of 26227 documents
Processed 17000 out of 26227 documents
Processed 18000 out of 26227 documents
Processed 19000 out of 26227 documents
Processed 20000 out of 26227 documents
Processed 21000 out of 26227 documents
Processed 22000 out of 26227 documents
Processed 23000 out of 26227 documents
Processed 24000 out of 26227 documents
Processed 25000 out of 26227 documents
Processed 26000 out of 26227 docum

In [22]:
from gensim.models.word2vec import Word2Vec

text_dim = 300
print("Training Word2Vec model...")
wordvec_model = Word2Vec(train_cleaned_word2vec, size=text_dim, window=5, min_count=3, workers=4, sg=1)
print("Word2Vec model created.")
print("%d unique words represented by %d dimensional vectors" % (len(wordvec_model.wv.vocab), text_dim))

Training Word2Vec model...
Word2Vec model created.
69813 unique words represented by 300 dimensional vectors


In [23]:
# Define function to create word vectors given a cleaned piece of text.
def create_average_vec(doc):
    average = np.zeros((text_dim,), dtype='float32')
    num_words = 0.
    for word in doc.split():
        if word in wordvec_model.wv.vocab:
            average = np.add(average, wordvec_model[word])
            num_words += 1.
    if num_words != 0.:
        average = np.divide(average, num_words)
    return average

In [24]:
# Create word vectors
train_cleaned_vec = np.zeros((df.shape[0], text_dim), dtype="float32")  # 19579 x 300
for i in range(len(train_cleaned)):
    train_cleaned_vec[i] = create_average_vec(train_cleaned[i])

print("Train word vector shape:", train_cleaned_vec.shape)

C:\Users\Tori\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


Train word vector shape: (26227, 300)


In [29]:
from sklearn.preprocessing import label_binarize

# Transform labels into one hot encoded format.
y_train_ohe = label_binarize(df['valid_score'], classes=[0, 1])
print('y_train_ohe shape: {}'.format(y_train_ohe.shape))

y_train_ohe shape: (26227, 1)


In [30]:
from sklearn.model_selection import train_test_split

# If using spaCy word vectors
# X_train, X_test, y_train, y_test = train_test_split(train_vec, y_train_ohe, test_size=0.2, random_state=21)

# If using Word2Vec word vectors
X_train, X_test, y_train, y_test = train_test_split(train_cleaned_vec, y_train_ohe, test_size=0.2, random_state=21)

print('X_train size: {}'.format(X_train.shape))
print('X_test size: {}'.format(X_test.shape))
print('y_train size: {}'.format(y_train.shape))
print('y_test size: {}'.format(y_test.shape))

X_train size: (20981, 300)
X_test size: (5246, 300)
y_train size: (20981, 1)
y_test size: (5246, 1)


In [46]:
#Load Libraries
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from scipy.stats import uniform
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import  scatter_matrix
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [50]:
#Linear Regression
kfold = KFold(n_splits=10, random_state=42,)
lin_reg= LinearRegression()
results_linreg= cross_val_score(lin_reg, train_cleaned_vec, y_train_ohe, cv=kfold)
print('Estimate accuracy',results_linreg.mean())

Estimate accuracy 0.04931561950776866


In [51]:
# Logistic Regression
kfold = KFold(n_splits=10, random_state=42)
logreg = LogisticRegression(solver='lbfgs',max_iter=10000)
results_logreg = cross_val_score(logreg, train_cleaned_vec, y_train_ohe, cv=kfold, scoring='accuracy')
print('Estimate accuracy', results_logreg.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate accuracy 0.8171021879152123


In [53]:
# Support Vector Machines
kfold = KFold(n_splits=10, random_state=42)
svc = SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False)
results_svc = cross_val_score(svc, train_cleaned_vec, y_train_ohe, cv=kfold,scoring='accuracy')
print('Estimate accuracy',results_svc.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate accuracy 0.6482103832406689


In [58]:
kfold = KFold(n_splits=10, random_state=42)
knn = KNeighborsClassifier(n_neighbors = 3)
results_knn = cross_val_score(knn, train_cleaned_vec, y_train_ohe, cv=kfold)
print('Estimate accuracy',results_knn.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was

Estimate accuracy 0.7487420003704831


In [59]:
kfold = KFold(n_splits=10, random_state=42)
gaussian = GaussianNB()
results_gaussian = cross_val_score(gaussian, train_cleaned_vec, y_train_ohe, cv=kfold)
print('Estimate accuracy',results_gaussian.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate accuracy 0.7009736669077424


In [60]:
kfold = KFold(n_splits=10, random_state=42)
perceptron = Perceptron(max_iter=1000,tol=1e-3)
results_perceptron = cross_val_score(perceptron, train_cleaned_vec, y_train_ohe, cv=kfold,scoring='accuracy')
print('Estimate accuracy',results_perceptron.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate accuracy 0.7335187493838611


In [61]:
kfold = KFold(n_splits=10, random_state=42)
linear_svc = LinearSVC(max_iter=1000)
results_linearsvc= cross_val_score(linear_svc, train_cleaned_vec, y_train_ohe, cv=kfold, scoring='accuracy')
print('Estimate accuracy', results_linearsvc.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate accuracy 0.8323899971879343


In [62]:
kfold = KFold(n_splits=10, random_state=42)
sgd = SGDClassifier(max_iter=1000,tol=1e-3)
results_sgd = cross_val_score(sgd, train_cleaned_vec, y_train_ohe, cv=kfold, scoring='accuracy')
print('Estimate accuracy',results_sgd.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate accuracy 0.7772599253275824


In [63]:
kfold = KFold(n_splits=10, random_state=42)
decision_tree = DecisionTreeClassifier()
results_decisiontree = cross_val_score(decision_tree, train_cleaned_vec, y_train_ohe, cv=kfold,scoring='accuracy')
print('Estimate accuracy',results_decisiontree.mean())

Estimate accuracy 0.6980623644675845


In [64]:
kfold = KFold(n_splits=10, random_state=42)
random_forest = RandomForestClassifier(n_estimators=100)
results_randomforest = cross_val_score(decision_tree, train_cleaned_vec, y_train_ohe, cv=kfold,scoring='accuracy')
print('Estimate accuracy',results_randomforest.mean())

Estimate accuracy 0.6934487152755665


In [65]:
kfold = KFold(n_splits=10, random_state=42)
clf = LinearDiscriminantAnalysis()
results_clf = cross_val_score(clf, train_cleaned_vec, y_train_ohe, cv=kfold, scoring='accuracy')
print('Estimate accuracy',results_clf.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate accuracy 0.8121085899452506


In [66]:
kfold = KFold(n_splits=10, random_state=42)
AB = AdaBoostClassifier()
results_AB = cross_val_score(AB, train_cleaned_vec, y_train_ohe, cv=kfold, scoring='accuracy')
print('Estimate accuracy',results_AB.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Estimate accuracy 0.7605198599608636


In [67]:
kfold = KFold(n_splits=10, random_state=42)
GBC = GradientBoostingClassifier()
results_GBC = cross_val_score(GBC, train_cleaned_vec, y_train_ohe, cv=kfold, scoring='accuracy')
print('Estimate accuracy',results_GBC.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

Estimate accuracy 0.7917866156714366


In [68]:
kfold = KFold(n_splits=10, random_state=42)
ETC=ExtraTreesClassifier(n_estimators=100)
results_ETC = cross_val_score(ETC, train_cleaned_vec, y_train_ohe, cv=kfold, scoring='accuracy')
print('Estimate accuracy',results_ETC.mean())

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was pa

Estimate accuracy 0.800595710130969


In [ ]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(train_cleaned_vec, y_train_ohe)
    grid_search.best_params_
    return grid_search.best_score_, grid_search.best_params_ ,grid_search.best_estimator_
svc_param_selection(train_cleaned_vec, y_train_ohe, 20)

def svc_param_selection2(X,y,nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    kernels = ['linear', 'poly', 'rbf', 'sigmoid']
    param_dist = {'C': Cs, 'gamma' : gammas, 'kernel': kernels}
    rand = RandomizedSearchCV(SVC(), param_dist, cv=nfolds, scoring='accuracy', n_iter=10, random_state=42)
    rand.fit(X,y)
    rand.best_score_
    rand.best_params_
    rand.best_estimator_
    return  rand.best_score_, rand.best_params_ ,rand.best_estimator_
svc_param_selection2(train_cleaned_vec, y_train_ohe,20)

C:\Users\Tori\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
